# Análise comparativa

O objetivo dessa eanalise é comparar modelos para encontrar um que possa ser melhor utilizado dentro do problema em questão (estimar a idade por meio do numero de aneis), para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

### 1.1 Configurações iniciais

Importações e configurações

In [53]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import os
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from pathlib import Path
from scipy import stats
from sklearn.preprocessing import StandardScaler

### 1.2 Obtendo dados

Nessa etapa vamos importar arquivos brutos de dados e o dicionário antes de começarmos o pre-processamento.

In [54]:
#Importando dados
names = ['Sexo', 'Comprimento', 'Diâmetro', 'Altura', 'Peso total', 'Peso sem concha', 'Peso do intestino', 'Peso da concha', 'Anéis']
df = pd.read_csv('../data/raw/abalone.csv',names = names, skiprows = 1)
dic = Path('../data/external/abalone_dict.csv')

Definindo os parametros do dataset de entrada e saida

In [55]:
# Fizemos aqui a um drop (exclusão temporaria) da coluna aneis para iniciar o modelo entrada (oque a maquina vai interpretar)
# e saida( oque ela vai tentar adivinhar)
X = df.drop('Anéis', axis=1)
Y = pd.DataFrame({'Anéis': df['Anéis']})

### 1.3 Tratamento de dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados

#### 1.3.1 - Tratamento de dados discrepantes

In [56]:
# Define valor de escala (outliers), valores acima disso serão descartados
z_score_threshold = 3

numeric_columns = X.select_dtypes(exclude=['object']).columns
X_numeric = df[numeric_columns]

z_scores = stats.zscore(X_numeric)
outliers = (abs(z_scores) > z_score_threshold).all(axis=1)

X_new = X[~outliers]

#### 1.3.2 - Conferencia de dados

In [57]:
# Checagem prévia
X_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Sexo               4176 non-null   object 
 1   Comprimento        4176 non-null   float64
 2   Diâmetro           4176 non-null   float64
 3   Altura             4176 non-null   float64
 4   Peso total         4176 non-null   float64
 5   Peso sem concha    4176 non-null   float64
 6   Peso do intestino  4176 non-null   float64
 7   Peso da concha     4176 non-null   float64
dtypes: float64(7), object(1)
memory usage: 293.6+ KB


Conforme verificado acima não houve remoção de nenhum valor

### 1.4 Divisão em conjuntos de treino e de teste

In [58]:
# Define os conjuntos que irão ser utilizados e suas caracteristicas
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

- Durante a analise notamos que não temos dados faltantes, então não será necessário fazer o preenchimento ou remoção dedestes valores

### 1.5 Codificação das variáveis categóricas

- exemplo
- exemplo

In [59]:
# Define a variavel para receber valor da LabelEncoder
le = LabelEncoder()

# Checagem de verificação
le.fit(X_train['Sexo'])

LabelEncoder()

In [60]:
# Treino
X_train['Sexo_t'] = le.transform(X_train['Sexo'])
X_train = X_train.drop('Sexo', axis=1)

In [61]:
# Teste
X_test['Sexo_t'] = le.transform(X_test['Sexo'])
X_test = X_test.drop('Sexo', axis=1)

### 1.6 Normalização dos dados

Iremos utilizar o metodo de normalização de dados chamado:

- Z-Score

In [64]:
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train)

In [65]:
X_test2 = scaler.fit_transform(X_test)

## Escolha do modelo

### 2.1 Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação cruzada por permutação, os modelos que serão testados serão:

- Logistic Regression (LR) - Regressão linear simples
- Regressão de Ridge (RR)
- K-Nearest-Neighbors (KNN)
- Support Vector Machine (SVM)

Além disso, cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Os modelos serão comparados através dos seguintes parâmetros

### 2.2 Configuração do experimento

Iremos separar o conjunto de dados em conjunto de testes e de treino para realizar a validação cruzada

Agora iremos realizar o teste com os modelos definidos anteriormente, testando também os diferentes parâmetros para obter a melhor combinação possível.

##### Regressão linear simples

In [66]:
# Atribuindo variáveis e realizando a execução
lr = LinearRegression()
lr.fit(X_train2, y_train)
y_pred_lr = lr.predict(X_test2)
y_pred_lr_int = np.round(y_pred_lr).astype(int)

In [67]:
# Checando os valores obtidos
mse_lr = mean_squared_error(y_test, y_pred_lr_int, squared=False)
print(f"RMSE: {mse_lr:.3f}")
r2_lr = r2_score(y_test,y_pred_lr_int)
print(f"R^2: {r2_lr:.3f}")

RMSE: 2.209
R^2: 0.559


#####  Regressão Ridge

- execução teste

In [70]:
# Parametros inicias
alphas = np.arange(0.25, 30, 0.25)

In [71]:
# Lógica da função de regressão de Ridge

best_alpha_5folds = 0
best_rmse_5folds = np.Inf

best_alpha_for_r2_5folds = 0
best_r2_5folds = 0

rmses_5folds = []
r2_scores_5folds = []

for alpha in alphas:
  ridge = Ridge(alpha= alpha)
  predictions = cross_val_predict(ridge, X_train2, y_train, cv=5)
  predictions_int = np.round(predictions).astype(int)
  rmse_score = mean_squared_error(y_train, predictions_int, squared=False)
  rmses_5folds.append(rmse_score)

  r2 = r2_score(y_train, predictions_int)
  r2_scores_5folds.append(r2)

  if rmse_score < best_rmse_5folds:
    best_rmse_5folds = rmse_score
    best_alpha_5folds = alpha

  if r2 > best_r2_5folds:
    best_r2_5folds = r2
    best_alpha_for_r2_5folds = alpha

print(f"Melhor lambda: {best_alpha_5folds}")
print(f"RMSE associado: {best_rmse_5folds:.3f}\n")

print(f"Lambda para melhor r2: {best_alpha_for_r2_5folds}")
print(f"Melhor r2: {best_r2_5folds:.3f}")

Melhor lambda: 3.25
RMSE associado: 2.269

Lambda para melhor r2: 3.25
Melhor r2: 0.496


- execução definitiva

In [74]:
#Parêmatros iniciais
ridge = Ridge(best_alpha_5folds)
ridge.fit(X_train2, y_train)

y_pred_ri = ridge.predict(X_test2)
y_pred_ri_int = np.round(y_pred_ri).astype(int)

In [75]:
#Função da regressão de Ridge
mse_ri = mean_squared_error(y_test, y_pred_ri_int, squared=False)
print(f"RMSE: {mse_ri:.3f}")
r2_ri= r2_score(y_test, y_pred_ri_int)
print(f"R^2: {r2_ri:.3f}")

RMSE: 2.217
R^2: 0.556


### 2.2 Configuração do experimento

Aqui iremos comparar os resultados e definir um modelo para ser utilizado no projeto

### 2.4 Resultado exxperimento

O melhor modelo obtido foi : *********

### 2.5 Percistencia do modelo